ДЗ:
1. На "отлично" - сдать все задачи
2. Сдавать в виде ссылок на git (не pull), на colab (не забыть расшарить права на чтение)
3. Сроки - лоялно (главное успеть к концу)

In [1]:
import pandas as pd
import numpy as np

Представим, что в магазине всего 10 товаров

In [2]:
recommended_list = [156, 1134, 991, 27, 1543, 3345, 533, 11, 43, 143] #id товаров
bought_list = [521, 32, 143, 991]

In [3]:
len(recommended_list)

10

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [4]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

# !!! ДЗ
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    hit_rate_at_k = (flags.sum() > 0) * 1
    
    
    return hit_rate_at_k

In [5]:
hit_rate(recommended_list, bought_list)

1

In [6]:
# для проверки поменяла в списке индексы местами и убрала с первого места релевантный
hit_rate_at_k(recommended_list, bought_list, 2) 

0

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [60, 400,  40, 40 , 90]  
flags = [1, 0,  0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [7]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision_at_k = flags.sum() / len(recommended_list)
        
    return precision_at_k


# !!! ДЗ
# для расчета money_precision_at_k добавим цены на рекомендованые товары
from random import randint

recommended_list_prices = [randint(100,400) for i in range(len(recommended_list))]
print('цены на рекомендованые товары',  recommended_list_prices)



def money_precision_at_k(recommended_list, bought_list, recommended_list_prices, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    recommended_list_prices = np.array(recommended_list_prices)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    relevant_money = np.dot(flags, recommended_list_prices) #стоимость релевантных товаров
    
    money_precision_at_k = relevant_money/sum(recommended_list_prices)
        
    return money_precision_at_k

цены на рекомендованые товары [306, 155, 390, 287, 304, 389, 237, 372, 274, 256]


In [8]:
precision(recommended_list, bought_list)

0.2

In [9]:
precision_at_k(recommended_list, bought_list, k=5)

0.2

In [10]:
money_precision_at_k(recommended_list, bought_list, recommended_list_prices, k=5)

0.27045769764216365

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [11]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


# !!! ДЗ

def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    recall_at_k = flags.sum()/len(bought_list)
    
    return recall_at_k


# !!! ДЗ
# для расчета money_recall_at_k добавим цены на купленные товары
# цена для релевантных товаров уже есть в массиве цен на рекомендованые товары

bought_list_prices = []
for i in bought_list:
    if i in recommended_list:  #если товар есть в рекомендованных, вытаскиваем его индекс и из списка цен берем цену по нему
        bought_list_prices.append(recommended_list_prices[(np.where(np.isin(recommended_list, i))[0])[0]]) 
    else:
        bought_list_prices.append(randint(100,400)) #если в рекомендациях не было товара, то заполняем произвольно
    
print('цены на купленные товары',  bought_list_prices)

def money_recall_at_k(recommended_list, bought_list, recommended_list_prices, bought_list_prices, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    recommended_list_prices = np.array(recommended_list_prices)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    relevant_money = np.dot(flags, recommended_list_prices)
    
    money_recall_at_k = relevant_money/sum(bought_list_prices)
    
    return money_recall_at_k

цены на купленные товары [302, 343, 256, 390]


In [12]:
recall(recommended_list, bought_list)

0.5

In [13]:
recall_at_k(recommended_list, bought_list, 5)

0.25

In [14]:
money_recall_at_k(recommended_list, bought_list, recommended_list_prices, bought_list_prices, k=5)

0.3020914020139427

---

# Метрики ранжирования

Если важен порядок рекомендаций. Подробнее можно почитать [здесь](https://habr.com/ru/company/econtenta/blog/303458/). Формулы в статье могут несколько отличаться от формул в лекции 

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [15]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [16]:
ap_k(recommended_list, bought_list, k=5)

0.16666666666666666

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [17]:
recommended_list_2 = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],] #id товаров
bought_list_2 = [[521, 32, 143, 991], [146, 29]]

In [18]:
num_users = len(bought_list)
num_users

4

In [19]:
def map_k(recommended_list, bought_list, k=5):
    
    num_users = len(bought_list)
    result = np.average([ap_k(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return result

In [20]:
map_k(recommended_list_2, bought_list_2, 5)

0.875

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


In [21]:
# DCG@k и NDCG@k без торча

import math

def ndcg_k(recommended_list, bought_list, k=5):

    flags = np.isin(recommended_list[:k], bought_list)
    
    #рассчитаем DCG    
    coeffs = [1/(i + 1) if i <= 2 else 1/math.log2(i + 1) for i in range(k)]
    
    dcg = np.dot(flags, coeffs)
    
    #Рассчитаем идеальный DCG
    ideal_dcg = sum([1/(i+1) if i<=2 else 1/math.log2(i+1) for i in range(k)])
    
    ndcg = dcg/ideal_dcg
    
    return ndcg

In [22]:
ndcg_k(recommended_list, bought_list)

0.12059773532998659

In [23]:
#для нескольких юзеров что-то пошло не так...не могу понять, почему не считаются флаги для двухмерных массивов

ndcg_k(recommended_list_2, bought_list_2, 10)

array([0., 0.])

#### Torch

In [122]:
import torch
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

In [123]:
ys_true

tensor([4, 3, 4, 0, 2])

In [124]:
# итоговый результат модели - отсортированный ys_pred
ys_pred

tensor([0.8877, 0.7190, 0.2051, 0.4511, 0.5632])

In [125]:
#ys_pred = np.array(ys_pred)

In [117]:
#ys_pred

In [126]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == "exp2":
        gain = 2 ** y_value - 1
    elif gain_scheme == "const":
        gain = y_value
    else:
        raise ValueError(f"{gain_scheme} method not supported, only exp2 and const.")
    return float(gain)

In [133]:
def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    ret = 0
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        gain = compute_gain(cur_y, gain_scheme)
        if idx < 1:
            ret += gain
        else: 
            ret += gain/math.log2(idx + 1)        
    return ret

In [134]:
dcg(ys_true, ys_pred, "const")

8.44020048965254

In [ ]:
def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
    pred_dcg = dcg(ys_true, ys_pred, gain_scheme)
    
    # your code ideal_dcg = что-то я запуталась(((
    
    ndcg = pred_dcg / ideal_dcg
    return ndcg

### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

[MRR](https://drive.google.com/file/d/1uMFKqWULFmehS0pJtvXKRygjYxQOPise/view?usp=sharing)

In [138]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    rec_relevant_indices = np.nonzero(np.isin(recommended_list[:k], bought_list))[0]
    if len(rec_relevant_indices) == 0:
        return 0
    rr = 1/(rec_relevant_indices[0]+1) #т к расчет идет с 1 (в отличии от индексов, которые с нуля)
    return rr

def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    
    num_users = len(bought_list)
    mrr = np.average([reciprocal_rank(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return mrr

In [139]:
reciprocal_rank(recommended_list, bought_list)

0.3333333333333333

# 3. Связь бизнес-метрик, ML-метрик и функции потерь(loss)

- **loss** - то, что оптимизирует модель (RMSE - Root Mean Squared Error)
- **ML-метрика** - то, как мы измеряем качество на test (money precision@5)
- **Бизнес-метрика** - то, что хочет оптимизировать бизнес (выручка)

В идеале loss = ML-метрика = бизнес-метрика, но это возмонжо в очень редких случаях

Надеемся, что:  
    *Оптимизация loss --> рост ML-метрик --> рост бизнес-метрик*

### Пример: рекомендательная система товаров на сайте  
      
На сайте есть баннер, на котором мы можем разместить 5 товаров. Надо порекомендовать эти 5 товаров персонально каждому юзеру

*Шаг 1: Определим бизнес-метрику*  
Бизнес хочет максимизировать выручку --> бизнес-метрика - **Выручка**  

*Шаг 2: Разложим ее на составляющие*  
Выручка =   
       Средний чек * кол-во покупок =   
       Средний чек * (число юзеров * конверсия из захода на сайт в заказ) =
       Число юзеров * (Средний чек * конверсия из захода на сайт в заказ)
       
Рекомендательная система влияет только на (Средний чек * конверсия из захода на сайт в заказ).   
Хорошим приближением этого является **money precision@5** - ML-метрика

*Шаг 3: loss*  
Это существенно сложнее. Персонализированные ML-модели не умеют напрямую оптимизировать конверсию. Стандартный loss - **RMSE**. Можно для начала попробовать его. 

P.S. Если вы хотите приблизить RMSE к money precision@5, то можно посчитать weighted RMSE, где вес каждого наблюдения = стоимость товара.   
P.S.S. Про другие виды loss будет рассказано в курсе